# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol);

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846873676070                   -0.70    5.0
  2   -7.852314096043       -2.26       -1.53    1.0
  3   -7.852646143155       -3.48       -2.52    3.5
  4   -7.852646678991       -6.27       -3.37    2.2
  5   -7.852646686057       -8.15       -4.78    1.2
  6   -7.852646686724       -9.18       -5.23    3.5
  7   -7.852646686730      -11.27       -6.11    1.0
  8   -7.852646686730      -12.54       -7.41    3.0
  9   -7.852646686730   +    -Inf       -7.50    2.0
 10   -7.852646686730      -15.05       -8.83    2.2
 11   -7.852646686730      -15.05       -8.79    1.0
 12   -7.852646686730   +  -14.45       -9.76    2.0
 13   -7.852646686730   +  -15.05       -9.95    2.0
 14   -7.852646686730      -14.35      -11.24    2.2
 15   -7.852646686730   +  -14.75      -11.85    3.0
 16   -7.852646686730   +  -15.05      -12.71    1.8


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846814302658                   -0.70           4.5
  2   -7.852524599266       -2.24       -1.64   0.80    2.2
  3   -7.852636012195       -3.95       -2.72   0.80    1.0
  4   -7.852646522993       -4.98       -3.25   0.80    2.0
  5   -7.852646681648       -6.80       -4.13   0.80    1.8
  6   -7.852646686380       -8.33       -4.91   0.80    1.5
  7   -7.852646686723       -9.46       -5.57   0.80    2.2
  8   -7.852646686729      -11.17       -6.37   0.80    1.5
  9   -7.852646686730      -12.30       -7.08   0.80    2.2
 10   -7.852646686730      -14.05       -7.95   0.80    1.5
 11   -7.852646686730      -14.75       -9.72   0.80    2.5
 12   -7.852646686730   +  -14.75       -9.30   0.80    3.8
 13   -7.852646686730      -14.75      -10.97   0.80    1.0
 14   -7.852646686730   +  -14.75      -10.69   0.80    2.0
 15   -7.852646686730      -15.05      -

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

Iter     Function value   Gradient norm 
     0     1.414216e+01     3.445215e+00
 * time: 0.3991730213165283
     1     1.378784e+00     1.787831e+00
 * time: 0.6104080677032471
     2    -1.481904e+00     1.973016e+00
 * time: 0.635200023651123
     3    -3.690843e+00     1.843058e+00
 * time: 0.6708741188049316
     4    -5.342740e+00     1.728917e+00
 * time: 0.7070169448852539
     5    -6.816305e+00     1.138249e+00
 * time: 0.7431221008300781
     6    -7.117328e+00     1.360281e+00
 * time: 0.7685749530792236
     7    -7.535303e+00     7.946468e-01
 * time: 0.7937829494476318
     8    -7.646024e+00     5.972287e-01
 * time: 0.8188009262084961
     9    -7.739171e+00     1.005552e-01
 * time: 0.8434560298919678
    10    -7.781923e+00     2.048528e-01
 * time: 0.8674120903015137
    11    -7.819388e+00     1.289662e-01
 * time: 0.891916036605835
    12    -7.838171e+00     5.759919e-02
 * time: 0.9161620140075684
    13    -7.845958e+00     3.809408e-02
 * time: 0.940504074096

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846832755302                   -0.70    4.5
  2   -7.852314099760       -2.26       -1.53    1.0


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686704                   -2.55
  2   -7.852646686730      -10.58       -5.94
  3   -7.852646686730   +    -Inf      -12.64


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 1.0647749742129399e-13
|ρ_newton - ρ_scfv| = 1.6897031228972678e-13
|ρ_newton - ρ_dm|   = 2.865237465154098e-10
